In [1]:
import torch
import torch.nn.functional as F
import torchvision.models.resnet as resnet

In [209]:

class ASPPpart(torch.nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation):
        super().__init__(
            torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, bias=False),
            torch.nn.BatchNorm2d(out_channels),
            torch.nn.ReLU(),
        )


class ASPP(torch.nn.Module):
    def __init__(self, in_channels, out_channels, rates=(3, 6, 9)):
        super().__init__()
        # TODO: Implement ASPP properly instead of the following
        modules = []
        rates = [2*x for x in rates]
        modules.append(ASPPpart(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1))
        for rate in rates:
            modules.append(ASPPpart(in_channels, out_channels, kernel_size=3, stride=1, padding=rate, dilation=rate))
       
        #global_avg = torch.nn.AdaptiveAvgPool2d(1) does not work gives [256,512, H, W] instead of [256,256, H,W]
        # therefore apply convolution with correct output channels
        global_avg = torch.nn.Sequential(torch.nn.AdaptiveAvgPool2d(1),
                                         torch.nn.Conv2d(in_channels, out_channels, kernel_size = 1))
        modules.append(global_avg)
        self.aspp_convs = torch.nn.ModuleList(modules)
        print("In channels", in_channels)
        print("Out channels", out_channels)
        # At this stage when called, already concatenated so we know how many out channels we have for each conv.
        # So total after concatenation of all diff layers of conv is len(self.aspp_convs)*out_channels
        self.conv_1x1 = torch.nn.Conv2d(out_channels*len(self.aspp_convs), out_channels, kernel_size = 1)

    def forward(self, x):
        # TODO: Implement ASPP properly instead of the following
        res = []                           
        features_h = x.size()[2] # is height of feature map image
        features_w = x.size()[3] # is width of feature map image
        for layer in self.aspp_convs:
            res.append(layer(x))
            print(layer)
            print("Tensor size = ", layer(x).size())
        #res[4] is the output of the average pooling but has h= 1, w = 1 so we upsample it to the needed height and width
        res[4] = F.interpolate(res[4],(features_h, features_w), mode = 'bilinear')
        res = torch.cat(res, dim = 1)
        print("after cat")
        return self.conv_1x1(res)

In [210]:
aspp = ASPP(512,256) # instanciate class

In channels 512
Out channels 256


In [211]:
features = torch.randn((256,512,10,10)) # format [BatchSize, Channels, Height, Width]
features_tasks = aspp(features) # call forward method
print("return size of forward = ", features_tasks.size())

ASPPpart(
  (0): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
Tensor size =  torch.Size([256, 256, 10, 10])
ASPPpart(
  (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
Tensor size =  torch.Size([256, 256, 10, 10])
ASPPpart(
  (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
Tensor size =  torch.Size([256, 256, 10, 10])
ASPPpart(
  (0): Conv2d(512, 256, kernel_size=(3, 3), stride=(1, 1), padding=(18, 18), dilation=(18, 18), bias=False)
  (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
)
Tensor size =  